In [1]:
#sympy for symbolic equations, latex2sympy for converting latex equations 

import sympy as sp
from latex2sympy2 import latex2sympy

### Core Parser Function

In [2]:
def parse_latex_to_sympy(latex_str: str):
    """
    Convert LaTeX math expression to SymPy object.

    Returns:
        sympy expression
    Raises:
        ValueError if parsing fails
    """
    try:
        expr = latex2sympy(latex_str)
        return sp.simplify(expr)
    except Exception as e:
        raise ValueError(f"Invalid LaTeX input: {e}")


In [3]:
examples = [
    r"x^2 + 2x + 1",
    r"\int_0^1 x^2 dx",
    r"\frac{d}{dx} x^3",
    r"\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix}"
]

for ex in examples:
    print("Input:", ex)
    print("Parsed:", parse_latex_to_sympy(ex))
    print()


Input: x^2 + 2x + 1
Parsed: x**2 + 2*x + 1

Input: \int_0^1 x^2 dx
Parsed: Integral(x**2, (x, 0, 1))

Input: \frac{d}{dx} x^3
Parsed: 3*x**2

Input: \begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix}
Parsed: Matrix([[1, 2], [3, 4]])



In [4]:
expr = parse_latex_to_sympy(r"\int_0^1 x^2 dx")
type(expr)

sympy.integrals.integrals.Integral

### Math Type Detection

In [5]:
import sympy as sp
from sympy import Integral, Derivative, Matrix, Eq

#### problem types

In [6]:
class ProblemType:
    ALGEBRA = "algebra_expression"
    EQUATION = "equation_solving"
    INTEGRAL = "calculus_integral"
    DERIVATIVE = "calculus_derivative"
    MATRIX = "linear_algebra_matrix"
    SYSTEM = "system_of_equations"
    UNKNOWN = "unknown"


In [7]:
def classify_problem(expr):
    """
    Classify SymPy expression into math category
    """

    # Integral
    if isinstance(expr, Integral):
        return ProblemType.INTEGRAL

    # Derivative
    if isinstance(expr, Derivative):
        return ProblemType.DERIVATIVE

    # Matrix
    if isinstance(expr, Matrix):
        return ProblemType.MATRIX

    # Equation
    if isinstance(expr, Eq):
        return ProblemType.EQUATION

    # System of equations
    if isinstance(expr, (list, tuple)) and all(isinstance(e, Eq) for e in expr):
        return ProblemType.SYSTEM

    # Default → algebra
    if isinstance(expr, sp.Expr):
        return ProblemType.ALGEBRA

    return ProblemType.UNKNOWN


In [8]:
from latex2sympy2 import latex2sympy

examples = [
    r"x^2 + 2x + 1",
    r"\int_0^1 x^2 dx",
    r"\frac{d}{dx} x^3",
    r"x^2 - 4 = 0",
    r"\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix}"
]

for ex in examples:
    expr = latex2sympy(ex)
    print(ex, "→", classify_problem(expr))


x^2 + 2x + 1 → algebra_expression
\int_0^1 x^2 dx → calculus_integral
\frac{d}{dx} x^3 → calculus_derivative
x^2 - 4 = 0 → system_of_equations
\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix} → linear_algebra_matrix


In [9]:
def build_problem_metadata(expr):
    return {
        "type": classify_problem(expr),
        "symbols": list(expr.free_symbols),
        "complexity": sp.count_ops(expr)
    }


In [10]:
expr = latex2sympy(r"\int_0^1 x^2 dx")
build_problem_metadata(expr)


{'type': 'calculus_integral', 'symbols': [], 'complexity': 2}

### Langchain Tools and Agents

In [22]:
def solve_algebra(expr):
    return sp.simplify(expr)

def solve_equation(expr):
    symbols = list(expr.free_symbols)
    return sp.solve(expr, symbols)

def solve_integral(expr):
    return sp.integrate(expr.function, *expr.limits)

def solve_derivative(expr):
    return sp.diff(expr.expr, *expr.variables)

def solve_matrix(expr):
    return expr.det()

In [ ]:
@tool
def algebra_tool(expression_str: str):
    """
    Solve an algebraic expression.
    """
    expr = sp.sympify(expression_str)
    return str(solve_algebra(expr))


@tool
def equation_tool(expression_str: str):
    """
    Solve an equation.
    """
    expr = sp.sympify(expression_str)
    return str(solve_equation(expr))


@tool
def integral_tool(expression_str: str):
    """
    Compute the integral of an expression.
    """
    expr = sp.sympify(expression_str)
    return str(solve_integral(expr))


@tool
def derivative_tool(expression_str: str):
    """
    Compute the derivative of an expression.
    """
    expr = sp.sympify(expression_str)
    return str(solve_derivative(expr))


@tool
def matrix_tool(expression_str: str):
    """
    Perform a matrix computation.
    """
    expr = sp.sympify(expression_str)
    return str(solve_matrix(expr))


In [25]:
tools = [
    algebra_tool,
    equation_tool,
    integral_tool,
    derivative_tool,
    matrix_tool
]

In [32]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model_name="openai/gpt-oss-120b", 
    temperature=0
)
agent = create_agent(
    model=llm,
    tools=tools,
)

In [34]:
def route_and_solve(expr, metadata):
    problem_type = metadata["type"]

    if problem_type == "calculus_integral":
        return integral_tool.run(str(expr))

    if problem_type == "calculus_derivative":
        return derivative_tool.run(str(expr))

    if problem_type == "equation_solving":
        return equation_tool.run(str(expr))

    if problem_type == "linear_algebra_matrix":
        return matrix_tool.run(str(expr))

    # Default to algebra simplification
    return algebra_tool.run(str(expr))

In [35]:
latex = r"\int_0^1 x^2 dx"
expr = latex2sympy(latex)
metadata = {"type": "calculus_integral"}

result = route_and_solve(expr, metadata)
print("Result:", result)

Result: 1/3
